# EBI Metadata - Preprocessing

In [1]:
import pandas as pd

metadata_ebi = pd.read_csv('../../data/results_sequence_tsv.txt',
                           sep='\t',
                           dtype={'collected_by': object,
                                  'collection_date': object,
                                  'culture_collection': object,
                                  'identified_by': object,
                                  'isolate': object,
                                  'isolation_source': object,
                                  'keywords': object,
                                  'lab_host': object,
                                  'location': object,
                                  'sample_accession': object,
                                  'strain': object,
                                  'study_accession': object})

metadata_ebi.info() # 51 columns, 2.8 mio entries

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2831557 entries, 0 to 2831556
Data columns (total 51 columns):
 #   Column                Dtype  
---  ------                -----  
 0   accession             object 
 1   altitude              float64
 2   base_count            int64  
 3   bio_material          float64
 4   cell_line             float64
 5   cell_type             float64
 6   collected_by          object 
 7   collection_date       object 
 8   country               object 
 9   cultivar              float64
 10  culture_collection    object 
 11  dataclass             object 
 12  description           object 
 13  dev_stage             float64
 14  ecotype               float64
 15  environmental_sample  bool   
 16  first_public          object 
 17  germline              bool   
 18  haplotype             float64
 19  host                  object 
 20  identified_by         object 
 21  isolate               object 
 22  isolation_source      object 
 23  keyword

## Remove Empty Columns

In [2]:
# Show how many NaN values each col has
metadata_ebi.isnull().sum()

accession                     0
altitude                2831557
base_count                    0
bio_material            2831557
cell_line               2831557
cell_type               2831557
collected_by            2748277
collection_date           66411
country                     462
cultivar                2831557
culture_collection      2831556
dataclass                     0
description                   0
dev_stage               2831557
ecotype                 2831557
environmental_sample          0
first_public                  0
germline                      0
haplotype               2831557
host                        451
identified_by           2831543
isolate                 1285827
isolation_source        1566587
keywords                1932985
lab_host                2831507
last_updated                  0
location                2828006
mating_type             2831557
metagenome_source       2831557
mol_type                      0
organelle               2831557
plasmid 

In [3]:
# Extract names of cols containing only NaN values
nan_cols = [i for i in metadata_ebi.columns if metadata_ebi[i].isnull().sum() == len(metadata_ebi)]
nan_cols

['altitude',
 'bio_material',
 'cell_line',
 'cell_type',
 'cultivar',
 'dev_stage',
 'ecotype',
 'haplotype',
 'mating_type',
 'metagenome_source',
 'organelle',
 'plasmid',
 'serotype',
 'serovar',
 'sex',
 'specimen_voucher',
 'sub_species',
 'sub_strain',
 'submitted_sex',
 'tissue_lib',
 'tissue_type',
 'variety']

In [4]:
# Reduce data frame to relevant 29 cols (6 columns do still contain a lot of NaN values)
metadata_ebi_relevant_cols = metadata_ebi.drop(nan_cols, axis=1)
metadata_ebi_relevant_cols.isnull().sum()

accession                     0
base_count                    0
collected_by            2748277
collection_date           66411
country                     462
culture_collection      2831556
dataclass                     0
description                   0
environmental_sample          0
first_public                  0
germline                      0
host                        451
identified_by           2831543
isolate                 1285827
isolation_source        1566587
keywords                1932985
lab_host                2831507
last_updated                  0
location                2828006
mol_type                      0
sample_accession         935723
scientific_name               0
sequence_md5                  0
sequence_version              0
strain                  2831400
study_accession          900293
tax_division                  0
tax_id                        0
topology                      0
dtype: int64

In [7]:
# Rename column and fill NaN values with empty strings
metadata_ebi_relevant_cols.rename({'country': 'country_and_region'}, axis=1, inplace=True)
metadata_ebi_relevant_cols['country_and_region'] = metadata_ebi_relevant_cols['country_and_region'].fillna('')

# Save as CSV
metadata_ebi_relevant_cols.to_csv('../../data/metadata_EBI_relevant_cols.csv', index=False)

## Data Cleaning

In [8]:
import pandas as pd

df = pd.read_csv('../../data/metadata_EBI_relevant_cols.csv',
                 dtype={'collected_by': object,
                        'collection_date': object,
                        'culture_collection': object,
                        'identified_by': object,
                        'isolate': object,
                        'isolation_source': object,
                        'keywords': object,
                        'lab_host': object,
                        'location': object,
                        'sample_accession': object,
                        'strain': object,
                        'study_accession': object})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2831557 entries, 0 to 2831556
Data columns (total 29 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   accession             object
 1   base_count            int64 
 2   collected_by          object
 3   collection_date       object
 4   country_and_region    object
 5   culture_collection    object
 6   dataclass             object
 7   description           object
 8   environmental_sample  bool  
 9   first_public          object
 10  germline              bool  
 11  host                  object
 12  identified_by         object
 13  isolate               object
 14  isolation_source      object
 15  keywords              object
 16  lab_host              object
 17  last_updated          object
 18  location              object
 19  mol_type              object
 20  sample_accession      object
 21  scientific_name       object
 22  sequence_md5          object
 23  sequence_version      int64 
 24

In [9]:
# TODO: Which columns are not relevant for further analysis?
cols_to_drop = ['scientific_name', # all 'SARS-CoV-2'
                'sequence_md5', # MD5 checksum should have to further meaning as it is a hash value
                'sequence_version', # nearly all version 1
                'study_accession', # name of study accession should have no influence on biases or does it encode e.g. country?
                'location', # same as country?
                'environmental_sample' #?
]

### Keywords

In [10]:
# Cleanup keyword column
df['keywords'].value_counts()

purposeofsampling:baselinesurveillance        894872
purposeofsampling:targetedefforts               1658
puposeofsampling:baselinesurveillance           1092
purposeofsampling:targeted_sequencing            611
purpose_of_sequencing:baselinesurveillance       214
purposeofsampling=targetedefforts                122
purposeofsampling=baselinesurveillance             3
Name: keywords, dtype: int64

In [11]:
df['keywords'].replace(['purposeofsampling:baselinesurveillance', 'puposeofsampling:baselinesurveillance', 'purpose_of_sequencing:baselinesurveillance', 'purposeofsampling=baselinesurveillance'], 'purpose_of_sampling:baseline_surveillance', inplace=True)
df['keywords'].replace(['purposeofsampling:targetedefforts', 'purposeofsampling=targetedefforts', 'purposeofsampling:targeted_sequencing'], 'purpose_of_sampling:targeted_efforts', inplace=True)
df['keywords'].value_counts()

purpose_of_sampling:baseline_surveillance    896181
purpose_of_sampling:targeted_efforts           2391
Name: keywords, dtype: int64

### Collection Date

In [5]:
# Check if start date of observations is useful
df['collection_date'] = pd.to_datetime(df['collection_date'])
df['collection_date'].min()  # -> first COVID-19 infections were in 2019, thus 2012 must be typo

Timestamp('2012-10-19 00:00:00')

In [13]:
# Check which other dates cannot exist
df[df['collection_date'].dt.year < 2020]['collection_date'].unique()

array(['2019-12-26T00:00:00.000000000', '2019-12-31T00:00:00.000000000',
       '2019-12-01T00:00:00.000000000', '2019-12-30T00:00:00.000000000',
       '2019-12-23T00:00:00.000000000', '2019-12-18T00:00:00.000000000',
       '2019-11-27T00:00:00.000000000', '2019-12-11T00:00:00.000000000',
       '2019-12-05T00:00:00.000000000', '2019-09-12T00:00:00.000000000',
       '2019-10-23T00:00:00.000000000', '2019-10-19T00:00:00.000000000',
       '2019-10-22T00:00:00.000000000', '2019-10-17T00:00:00.000000000',
       '2019-10-12T00:00:00.000000000', '2019-11-22T00:00:00.000000000',
       '2019-12-15T00:00:00.000000000'], dtype='datetime64[ns]')

In [11]:
# Remove same "2012"-typo for 23 samples in collection date by exchanging with "2021"
df['collection_date'].replace('2012-10-19', '2021-10-19', inplace=True)

## Data Enrichment

### Gender

In [12]:
df['host'].value_counts()

Homo sapiens                                                       2812197
homo sapiens                                                          2397
Homo sapiens; FEMALE                                                  1987
Homo sapiens; MALE                                                    1758
Homo sapiens; UNKNOWN                                                  151
                                                                    ...   
Homo sapiens; Female, age 21,Asian;White,Not Hispanic or Latino          1
Homo sapiens; Female, age 20,Asian,Not Hispanic or Latino                1
Homo sapiens; Female, age 64,White,Not Hispanic or Latino                1
Homo sapiens; Female, age 55,White,Not Hispanic or Latino                1
Homo sapiens; age 440                                                    1
Name: host, Length: 1402, dtype: int64

In [13]:
def get_gender(host_value):
    if any([substring in host_value.lower() for substring in ['female', 'femle', 'gender: f']]):
        return 'female'
    if any([substring in host_value.lower() for substring in ['male', 'gender: m']]):
        return 'male'
    else:
        return 'unknown'

In [26]:
# Add gender column
df['gender'] = df.apply(lambda row: get_gender(str(row['host'])), axis=1)
df['gender'].value_counts()

unknown    2817207
female        7611
male          6739
Name: gender, dtype: int64

In [27]:
# Verify if all gender descriptions are catched
df_unknown_gender = df[df['gender'] == 'unknown']

pd.set_option("display.max_rows", None)
df_unknown_gender['host'].value_counts()

Homo sapiens                           2812197
homo sapiens                              2397
Homo sapiens; UNKNOWN                      151
Neovison vison                              62
Homo sapiens; age 19                        55
Mink                                        54
Homo sapiens; age 21                        40
Homo sapiens; age 20                        38
Homo sapiens; age 18                        35
Homo sapiens; age 50                        35
Homo sapiens; age 49                        29
Homo sapiens; age 22                        28
Homo sapiens; age 47                        27
Homo sapiens; age 37                        25
Homo sapiens; age 59                        24
Homo sapiens; age 38                        23
Homo sapiens; age 58                        23
Homo sapiens; age 40                        23
Homo sapiens; age 60                        23
Homo sapiens; age 56                        22
Homo sapiens; age 42                        21
Homo sapiens;

### Age

In [28]:
# Manually fix typos in "host" column
df['host'] = df['host'].replace(['agee'], ['age'], regex=True)

In [29]:
import numpy as np

def get_age(host_value):
    if 'age' in host_value.lower():
        if 'younger than' in host_value.lower():
            # example: "age Younger Than 15"
            return np.nan
        elif 'day' in host_value.lower():
            age_in_days = int(host_value[(host_value.lower().find('day') - 3): (host_value.lower().find('day') - 1)])
            # example: "age 42 day/s"
            return round(age_in_days / 365, 2)
        elif 'months' in host_value.lower():
            age_in_months = int(
                host_value[(host_value.lower().find('months') - 3): (host_value.lower().find('months') - 1)])
            # example: "age 6 months"
            return round(age_in_months / 12, 2)
        elif 'mos' in host_value.lower():
            if host_value[host_value.lower().find('mos') - 1] == ' ':
                age_in_months = int(
                    host_value[(host_value.lower().find('mos') - 3): (host_value.lower().find('mos') - 1)])
                # example: "age 6 mos/MOS"
                return round(age_in_months / 12, 2)
            else:
                age_in_months = int(host_value[(host_value.lower().find('mos') - 2): (host_value.lower().find('mos'))])
                # example: "age 6MOS"
                return round(age_in_months / 12, 2)
        elif host_value[host_value.lower().find('age') + 3] == ' ':
            if 'age unknown' in host_value.lower():
                # example: "age Unknown"
                return np.nan
            else:
                # example: "age 42"
                return host_value[(host_value.lower().find('age') + 4): (host_value.lower().find('age') + 6)]
        elif host_value[host_value.lower().find('age') + 3] == ':':
            if host_value[host_value.lower().find('age') + 4] == ' ':
                # example: "age: 42"
                return host_value[(host_value.lower().find('age') + 5): (host_value.lower().find('age') + 7)]
            else:
                # example: "age:42"
                return host_value[(host_value.lower().find('age') + 4): (host_value.lower().find('age') + 6)]
        else:
            # example: "age42"
            return host_value[(host_value.lower().find('age') + 3): (host_value.lower().find('age') + 5)]
    elif 'year old' in host_value.lower():
        # example: "42 year old"
        return host_value[(host_value.lower().find('year old') - 3): (host_value.lower().find('year old') - 1)]
    else:
        return np.nan

In [30]:
# Add age column (measured in years)
df['age'] = df.apply(lambda row: get_age(str(row['host'])), axis=1)

### Country

In [31]:
# Extract country from "country_and_region" column
df['country_and_region'] = df['country_and_region'].astype(str)
df['country'] = [country_and_region.split(':')[0] for country_and_region in df['country_and_region']]
df['country'].value_counts()

USA                   1389589
United Kingdom        1075204
Germany                210968
Switzerland             91095
Australia               13311
Slovakia                 9548
Iceland                  5365
Bahrain                  3304
Mexico                   2949
Kenya                    2709
New Zealand              2427
India                    2062
Estonia                  1681
France                   1618
Netherlands              1562
Poland                   1219
Spain                    1184
Thailand                 1106
Saudi Arabia             1090
Egypt                    1071
Brazil                   1018
Japan                     942
Lebanon                   931
Pakistan                  827
Bangladesh                666
Seychelles                541
nan                       462
China                     423
Iran                      416
Chile                     383
Nigeria                   383
Russia                    360
Hong Kong                 347
Italy     

In [32]:
df['country'].unique()

array(['Japan', 'United Kingdom', 'Spain', 'Switzerland', 'France',
       'Poland', 'Estonia', 'Philippines', 'nan', 'Bangladesh', 'China',
       'Sweden', 'Netherlands', 'Germany', 'Zimbabwe', 'Italy',
       'Thailand', 'USA', 'Australia', 'India', 'Finland', 'South Korea',
       'Malaysia', 'Taiwan', 'Belgium', 'Nepal', 'Hong Kong', 'Brazil',
       'Viet Nam', 'Iran', 'Nigeria', 'Georgia', 'Pakistan', 'Colombia',
       'Peru', 'Israel', 'Iraq', 'Tunisia', 'Singapore', 'South Africa',
       'Turkey', 'Greece', 'Serbia', 'Sri Lanka', 'Czech Republic',
       'Puerto Rico', 'Kazakhstan', 'Uzbekistan', 'Guam', 'Uruguay',
       'Jamaica', 'Russia', 'Egypt', 'Morocco', 'Kenya', 'Timor-Leste',
       'Saudi Arabia', 'Benin', 'Chile', 'New Zealand', 'Mexico',
       'Bahrain', 'Zambia', 'Lebanon', 'Jordan', 'Belize', 'Guatemala',
       'Sierra Leone', 'Cuba', 'Ghana', 'Venezuela', 'Denmark', 'Myanmar',
       'Libya', 'Malta', 'Gabon', 'Romania', 'West Bank', 'Ecuador',
       'Cana

### Number of Inhabitants

In [33]:
import json

df['country'] = df['country'].replace(np.nan, 'Unknown')

# Load number of inhabitants per country (taken from Wikipedia)
with open('../helper_dicts/country_inhabitants.json') as f:
    country_inhabitants_map = json.load(f)

In [34]:
# Add column with number of inhabitants
df['n_inhabitants'] = df['country'].map(country_inhabitants_map)
df['n_inhabitants'].describe()

count    2.830840e+06
mean     1.967880e+08
std      1.385421e+08
min      1.790700e+04
25%      6.708100e+07
50%      8.312928e+07
75%      3.314493e+08
max      1.418120e+09
Name: n_inhabitants, dtype: float64

### Region

In [35]:
import numpy as np

def get_region(country_and_region):
    if country_and_region.split(':', 1) != [country_and_region]:
        if country_and_region.split(':', 1)[1][0] == ' ':
            # Remove whitespace in front of region
            return country_and_region.split(':', 1)[1][1:]
        else:
            return country_and_region.split(':', 1)[1]
    else:
        # Insert NaN if only country is available in "country_and_region" column
        return np.nan

# Add region column
df['region'] = df.apply(lambda row: get_region(str(row['country_and_region'])), axis=1)

In [36]:
# TODO: Derive race from "host" column and add as new column

In [12]:
df.to_csv('../../data/metadata_EBI_preprocessed.csv', index=False)